In [1]:
from fastai2.vision.all import *
from fastai2.vision.widgets import *
import requests
from http import HTTPStatus
from io import BytesIO

In [21]:
def load_remote_model(model_url:str):
    response = requests.get(model_url)
    if response.status_code != HTTPStatus.OK:
        print(f"Error Downloading Model, status: {response.status_code}")
        return None
    return load_learner(BytesIO(response.content), cpu=True)

def load_remote_img_bytes(img_url:str):
    try:
        response = requests.get(img_url)
    except Exception as e:
        print(f"Error while loading image: {e}")
        return None
    if response.status_code != HTTPStatus.OK:
        print(f"Error Downloading Img, status: {response.status_code}")
        return None
    img = Image.open(BytesIO(response.content))
    buf = BytesIO()
    img_url = img_url.lower()
    if img_url.endswith('jpg') or img_url.endswith('jpeg'):
        img.save(buf, format='JPEG')
    elif img_url.endswith('png'):
        img.save(buf, format='PNG')
    elif img_url.endswith('webp'):
        img.save(buf, format='WEBP')
    elif img_url.endswith('bmp'):
        img.save(buf, format='BMP')
    elif img_url.endswith('gif'):
        img.save(buf, format='GIF')
    elif img_url.endswith('tiff'):
        img.save(buf, format='TIFF')
    else:
        print("Reading as a JPEG Image")
        img.save(buf, format='JPEG')
    return buf.getvalue()

In [3]:
model_url = "https://mobile-random-stuff.s3.amazonaws.com/tennis_with_clean_01.pkl"
learn_model = load_remote_model(model_url)

In [22]:
txt = widgets.Text(value="", placeholder="", disabled=False)
btn_classify = widgets.Button(description='Classify')
btn_upload = widgets.FileUpload()
label1 = widgets.Label(value="Enter Image URL Below")
label2 = widgets.Label(value="OR UPLOAD an Image: jpeg, png, webp only")
out_pl = widgets.Output()
lbl_pred = widgets.Label(value="Out Results")

In [23]:
def add_image_pred(img):
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(256,256))
    lbl_pred.value = "Running Prediction Model!"    
    pred,pred_idx,probs = learn_model.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [24]:
def on_upload(change):
    lbl_pred.value = "Computing the Inference"
    img = PILImage.create(btn_upload.data[-1])
    add_image_pred(img)

In [25]:
def on_classify_click(change):
    url = txt.value
    lbl_pred.value = "Computing the Inference"
    data = load_remote_img_bytes(url)
    if data:
        img = PILImage.create(data)
        add_image_pred(img)

In [26]:
btn_upload.observe(on_upload, names=['data'])
btn_classify.on_click(on_classify_click)

## The Amazing Tennis Classifier!
Enter an Image URL or Upload an Image to classify images into 4 different tennis actions: Forehand, Backhand, Serve, Smash. The Images should be JPEG, PNG, WEBP, BMP, TIFF or GIF.  

In [27]:
VBox([label1, HBox([txt, btn_classify]), label2, btn_upload, out_pl, lbl_pred])

Error while loading image: Invalid URL 'bdodu': No schema supplied. Perhaps you meant http://bdodu?
